## 📖 Teoria: PRNG (Pseudo-Random Number Generator)

### rand() e srand()

```c
#include <stdlib.h>
#include <time.h>

// Inizializza seed (una volta all'avvio)
srand(time(NULL));
      └───┬────┘
      timestamp corrente (secondi da 1970)

// Genera numero casuale [0, RAND_MAX]
int r = rand();
```

### Generare uint32_t casuale

```
RAND_MAX tipicamente = 32767 (15 bit)
uint32_t richiede 32 bit

Soluzione: combina più chiamate rand()

uint32_t = (rand() << 16) | rand()
           └────┬────┘       └──┬──┘
           16 bit alti    16 bit bassi
```

### Perché time(NULL)?

```
Senza seed:
  Run 1: IP sequence A
  Run 2: IP sequence A  ← Stesso!
  Run 3: IP sequence A

Con time(NULL) seed:
  Run 1 (t=1000): sequence A
  Run 2 (t=1001): sequence B
  Run 3 (t=1002): sequence C  ← Diverso!
```

## 📖 Algoritmo: Genera IP con Vincoli

```
generate_public_ip():
  loop:
    ip = random_uint32()
    
    if get_ip_type(ip) == PUBLIC:
      return ip
    
    // Retry se privato/riservato
```

### Con Subnet:

```
generate_in_subnet(network, netmask):
  host_bits = 32 - prefix_len
  max_host = (1 << host_bits) - 1
  
  loop:
    host_part = rand() % max_host
    ip = network | host_part
    
    if not is_network_or_broadcast(ip):
      return ip
```

## 📝 Implementazione

In [ ]:
%%bash
mkdir -p /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/07
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/07

# Copia tutti i file precedenti
cp ../06/*.h ../06/*.c .
rm -f test_*.c  # Rimuovi test vecchi

# Crea generator.h
cat > generator.h << 'EOF'
#ifndef GENERATOR_H
#define GENERATOR_H

#include <stdint.h>
#include <stdbool.h>
#include "ipset.h"
#include "parser.h"

/**
 * Inizializza RNG
 * Deve essere chiamato una volta all'avvio
 */
void init_random();

/**
 * Genera uint32_t casuale (full range)
 */
uint32_t random_uint32();

/**
 * Genera IP casuale pubblico
 * Escludi riservati e privati
 * 
 * @param max_retries Max tentativi prima di fallire
 * @param success Output: true se generato, false se timeout
 * @return IP pubblico casuale
 */
uint32_t generate_public_ip(int max_retries, bool *success);

/**
 * Genera IP in subnet specifica
 * 
 * @param network Network address
 * @param netmask Netmask
 * @param max_retries Max tentativi
 * @param success Output: true se generato
 * @return IP nella subnet (esclude network e broadcast)
 */
uint32_t generate_ip_in_subnet(uint32_t network, 
                                uint32_t netmask,
                                int max_retries, 
                                bool *success);

/**
 * Genera N IP univoci pubblici
 * 
 * @param count Numero IP da generare
 * @param set Output: set con IP generati
 * @return Numero IP effettivamente generati
 */
size_t generate_unique_ips(size_t count, IPSet *set);

/**
 * Genera IP in subnet da specifica
 */
size_t generate_ips_from_spec(const IPSpec *spec, 
                               size_t count, 
                               IPSet *set);

#endif // GENERATOR_H
EOF

echo "✅ generator.h creato"

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/07

cat > generator.c << 'EOF'
#include "generator.h"
#include "ip_utils.h"
#include <stdlib.h>
#include <time.h>

/**
 * Inizializza RNG con timestamp
 */
void init_random() {
    srand((unsigned int)time(NULL));
}

/**
 * Genera uint32_t casuale completo
 * 
 * rand() tipicamente genera 15-16 bit
 * Combina 2 chiamate per 32 bit
 */
uint32_t random_uint32() {
    // High 16 bit
    uint32_t high = (uint32_t)rand() & 0xFFFF;
    
    // Low 16 bit
    uint32_t low = (uint32_t)rand() & 0xFFFF;
    
    return (high << 16) | low;
}

/**
 * Genera IP pubblico (no riservati, no privati)
 * 
 * Algoritmo: retry finché non pubblico
 */
uint32_t generate_public_ip(int max_retries, bool *success) {
    for (int i = 0; i < max_retries; i++) {
        uint32_t ip = random_uint32();
        
        // Filtra tipo
        if (get_ip_type(ip) == IP_PUBLIC) {
            if (success) *success = true;
            return ip;
        }
    }
    
    // Timeout
    if (success) *success = false;
    return 0;
}

/**
 * Genera IP in subnet
 * 
 * Algoritmo:
 * 1. Calcola quanti bit host
 * 2. Genera host part casuale
 * 3. Combina con network part
 * 4. Escludi network e broadcast
 */
uint32_t generate_ip_in_subnet(uint32_t network, 
                                uint32_t netmask,
                                int max_retries, 
                                bool *success) {
    // Calcola host bits
    uint32_t host_mask = ~netmask;
    uint32_t max_host = host_mask;
    
    if (max_host < 2) {
        // /31 o /32: nessun host valido
        if (success) *success = false;
        return 0;
    }
    
    uint32_t broadcast = network | host_mask;
    
    for (int i = 0; i < max_retries; i++) {
        // Host part casuale [0, max_host]
        uint32_t host_part = random_uint32() & host_mask;
        uint32_t ip = network | host_part;
        
        // Escludi network e broadcast
        if (ip != network && ip != broadcast) {
            if (success) *success = true;
            return ip;
        }
    }
    
    if (success) *success = false;
    return 0;
}

/**
 * Genera N IP univoci pubblici
 * 
 * Algoritmo:
 * 1. Loop fino a count IP
 * 2. Genera IP pubblico
 * 3. ipset_add (auto-dedup)
 * 4. Se duplicato, retry
 */
size_t generate_unique_ips(size_t count, IPSet *set) {
    if (!set) return 0;
    
    size_t generated = 0;
    int consecutive_failures = 0;
    const int MAX_FAILURES = 1000;
    
    while (generated < count && consecutive_failures < MAX_FAILURES) {
        bool success;
        uint32_t ip = generate_public_ip(100, &success);
        
        if (!success) {
            consecutive_failures++;
            continue;
        }
        
        // ipset_add ritorna false se duplicato
        if (ipset_add(set, ip)) {
            generated++;
            consecutive_failures = 0;  // Reset
        } else {
            consecutive_failures++;
        }
    }
    
    return generated;
}

/**
 * Genera IP da specifica (CIDR/Range/Single)
 */
size_t generate_ips_from_spec(const IPSpec *spec, 
                               size_t count, 
                               IPSet *set) {
    if (!spec || !set) return 0;
    
    if (spec->type == SPEC_CIDR) {
        // Genera in subnet
        uint32_t netmask = create_netmask(spec->prefix_len);
        
        size_t generated = 0;
        int failures = 0;
        
        while (generated < count && failures < 1000) {
            bool success;
            uint32_t ip = generate_ip_in_subnet(
                spec->start_ip, netmask, 100, &success);
            
            if (success && ipset_add(set, ip)) {
                generated++;
                failures = 0;
            } else {
                failures++;
            }
        }
        
        return generated;
    } else {
        // Range/Single: espandi a set, poi prendi random
        IPSet *pool = ipset_create(256);
        expand_spec_to_set(spec, pool);
        
        size_t pool_size = ipset_size(pool);
        size_t to_gen = count < pool_size ? count : pool_size;
        
        for (size_t i = 0; i < to_gen; i++) {
            // Prendi IP casuale dal pool
            size_t idx = random_uint32() % pool_size;
            uint32_t ip = ipset_get(pool, idx);
            ipset_add(set, ip);
        }
        
        ipset_free(pool);
        return to_gen;
    }
}
EOF

echo "✅ generator.c implementato"

## ✅ Test Generator

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/07

cat > test_generator.c << 'EOF'
#include <stdio.h>
#include "generator.h"
#include "ip_utils.h"

void test_random_uint32() {
    printf("=== TEST random_uint32() ===\n");
    
    init_random();
    
    printf("\n  10 numeri casuali:\n");
    for (int i = 0; i < 10; i++) {
        uint32_t r = random_uint32();
        printf("    %10u (0x%08X)\n", r, r);
    }
    printf("\n");
}

void test_public_ip() {
    printf("=== TEST generate_public_ip() ===\n\n");
    
    init_random();
    
    printf("  Genera 10 IP pubblici:\n");
    for (int i = 0; i < 10; i++) {
        bool success;
        uint32_t ip = generate_public_ip(1000, &success);
        
        if (success) {
            char buf[16];
            ip_to_string(ip, buf);
            
            IPType type = get_ip_type(ip);
            const char *type_str = ip_type_to_string(type);
            
            printf("    [%d] %-15s → %s\n", i, buf, type_str);
        } else {
            printf("    [%d] ✗ Failed\n", i);
        }
    }
    printf("\n");
}

void test_subnet_generation() {
    printf("=== TEST generate_ip_in_subnet() ===\n\n");
    
    init_random();
    
    // Test subnet: 192.168.1.0/24
    uint32_t network;
    string_to_ip("192.168.1.0", &network);
    uint32_t netmask = create_netmask(24);
    
    printf("  Subnet: 192.168.1.0/24\n\n");
    printf("  Genera 10 IP nella subnet:\n");
    
    for (int i = 0; i < 10; i++) {
        bool success;
        uint32_t ip = generate_ip_in_subnet(network, netmask, 100, &success);
        
        if (success) {
            char buf[16];
            ip_to_string(ip, buf);
            
            // Verifica in subnet
            bool in_sub = ip_in_subnet(ip, network, netmask);
            
            printf("    [%d] %-15s %s\n", i, buf, 
                   in_sub ? "✓" : "✗ Fuori subnet!");
        }
    }
    printf("\n");
}

void test_unique_generation() {
    printf("=== TEST generate_unique_ips() ===\n\n");
    
    init_random();
    
    IPSet *set = ipset_create(64);
    
    printf("  Genera 20 IP univoci...\n");
    size_t generated = generate_unique_ips(20, set);
    
    printf("  Generati: %zu\n", generated);
    printf("  Set size: %zu\n\n", ipset_size(set));
    
    printf("  Primi 5 IP:\n");
    for (size_t i = 0; i < 5 && i < ipset_size(set); i++) {
        char buf[16];
        ip_to_string(ipset_get(set, i), buf);
        printf("    [%zu] %s\n", i, buf);
    }
    
    ipset_free(set);
    printf("\n");
}

void test_spec_generation() {
    printf("=== TEST generate_ips_from_spec() ===\n\n");
    
    init_random();
    
    // Test CIDR
    IPSpec spec;
    parse_ip_spec("10.0.0.0/28", &spec);
    
    IPSet *set = ipset_create(32);
    
    printf("  Genera 10 IP da 10.0.0.0/28...\n");
    size_t count = generate_ips_from_spec(&spec, 10, set);
    
    printf("  Generati: %zu\n\n", count);
    
    printf("  IP generati:\n");
    for (size_t i = 0; i < ipset_size(set); i++) {
        char buf[16];
        ip_to_string(ipset_get(set, i), buf);
        printf("    [%zu] %s\n", i, buf);
    }
    
    ipset_free(set);
    printf("\n");
}

void test_distribution() {
    printf("=== TEST Distribuzione Classi ===\n\n");
    
    init_random();
    
    IPSet *set = ipset_create(256);
    generate_unique_ips(100, set);
    
    // Conta per classe
    int counts[5] = {0};  // A, B, C, D, E
    
    for (size_t i = 0; i < ipset_size(set); i++) {
        uint32_t ip = ipset_get(set, i);
        IPClass class = classify_ip(ip);
        counts[class]++;
    }
    
    printf("  Su 100 IP pubblici generati:\n\n");
    printf("    Classe A: %d\n", counts[CLASSE_A]);
    printf("    Classe B: %d\n", counts[CLASSE_B]);
    printf("    Classe C: %d\n", counts[CLASSE_C]);
    printf("    Classe D: %d (multicast)\n", counts[CLASSE_D]);
    printf("    Classe E: %d (riservato)\n", counts[CLASSE_E]);
    
    ipset_free(set);
}

int main() {
    test_random_uint32();
    test_public_ip();
    test_subnet_generation();
    test_unique_generation();
    test_spec_generation();
    test_distribution();
    
    printf("\n✅ Tutti i test completati!\n");
    return 0;
}
EOF

gcc -std=c11 -Wall -Wextra -o test_generator \
    ip_utils.c ipset.c parser.c generator.c test_generator.c && ./test_generator

## 🔍 Analisi: Collision Handling

### Problema: Duplicati

```
Genera 1000 IP da pool di 256:
  Dopo 256 IP univoci, TUTTI i successivi sono duplicati!
  
  Senza timeout → loop infinito
```

### Soluzione: Failure Counter

```c
int consecutive_failures = 0;
const int MAX_FAILURES = 1000;

while (generated < count && 
       consecutive_failures < MAX_FAILURES) {
    
    if (ipset_add(set, ip)) {
        generated++;
        consecutive_failures = 0;  // Reset!
    } else {
        consecutive_failures++;
    }
}
```

**Perché reset?**
- Se trova IP univoco → pool non esaurito
- Reset counter per dare più chance

## 📊 Statistiche Generazione

### Spazio IP Pubblici

```
Totale IPv4: 2^32 = 4,294,967,296

Riservati:
  0.0.0.0/8      →   16,777,216
  127.0.0.0/8    →   16,777,216
  224.0.0.0/4    →  268,435,456 (multicast)
  240.0.0.0/4    →  268,435,456 (classe E)

Privati (RFC 1918):
  10.0.0.0/8     →   16,777,216
  172.16.0.0/12  →    1,048,576
  192.168.0.0/16 →       65,536

Pubblici disponibili: ~3.7 miliardi
```

### Probabilità Collision

```
Birthday paradox:
  Con 65536 IP generati → 50% prob di duplicato
  Con 1M IP → alta probabilità collision
  
Soluzione RANDIP: IPSet dedup + failure counter
```

## 📚 Recap

### ✅ Generator completo:

**Funzioni core:**
1. `init_random()` - Seed con time()
2. `random_uint32()` - PRNG full 32-bit
3. `generate_public_ip()` - Filtro tipo + retry
4. `generate_ip_in_subnet()` - Network + host random
5. `generate_unique_ips()` - Batch con dedup
6. `generate_ips_from_spec()` - Da CIDR/Range/Single

### 🎓 Concetti:

- **PRNG**: `rand()` + `srand(time())`
- **Seeding**: timestamp per variabilità
- **32-bit random**: combina 2x rand()
- **Retry pattern**: loop con max_retries
- **Collision handling**: failure counter
- **Subnet generation**: network | (rand() & host_mask)

### 🔑 Formule:

```c
// Random 32-bit
uint32_t r = (rand() << 16) | rand();

// IP in subnet
uint32_t host = rand() & ~netmask;
uint32_t ip = network | host;

// Dedup check
if (ipset_add(set, ip)) { /* nuovo */ }
```

### ➡️ Prossimi Notebook

**Notebook 8: CLI** - getopt, ProgramOptions, usage  
**Notebook 9: Main** - Integrazione tutto, output file

---

## 🎉 Notebook 7 Completato!

**Funzioni**: 6 generatori + RNG  
**Concetti**: PRNG, retry, collision, dedup

Continua con Notebook 8 (CLI)! 🚀